<a href="https://colab.research.google.com/github/neurall/PlenoxelsColab/blob/main/Plenoxels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This first version seems to work at least on Colab Pro providing 16g gpu and with high mem option enabled(32g ram) to fit and train 512 size of voxel cube (Colab Pro is 10$ a month which is price of 1 hamburger)  or connect colab to better gpu and ram via gcloud https://drive.google.com/file/d/1ijawhI6AuPXxWhLu8flkrHZPTu3o_zF0/view
<br>

If you want. You can run it even on your local 10g 3080. Just delete in configs [512,512,512] that is there for 16g gpus to run in consumer gpus [256,256,256]= max 10g gpu mem usage.  
Results will be much lower res. But at least you can now play with it locally now.
<br>

You can fork this colab at https://github.com/neurall/PlenoxelsColab/blob/main/Plenoxels.ipynb

It is an attempt to make running in Colab cloud GPU possible for the wonderfull breakthrough that is Plenoxels made by fantastic team of incredibly talented  

# Plenoxels Authors: (Alex Yu * Sara Fridovich-Keil * Matthew Tancik Qinhong Chen Benjamin Recht Angjoo Kanazawa UC Berkeley * Equal contribution)  

Github:            https://github.com/sxyu/svox2  
Website and video: https://alexyu.net/plenoxels  
arXiv:             https://arxiv.org/abs/2112.05131  

Their achievement of shortening voxel extraction from days to minutes is nothing short of extreaordinary. Incredible feat and wonderfull insight.

But due to recent shortage of gpus on market.  
<br>
Simple instant and easily accesible GPU cloud based workflow now seems to be more important to fast research iteration than ever.  
<br>
This colab will currently sadly not yet work on free colab with voxel size 512

Perhaps we can train four quadrants in [256,256,256]  voxel sub cubes in separate passes instead of one [512,512,512] pass in config? But that would require change scene center and bounds somehow for each pass ?

# Free colab has just 12g ram 11g (P8) gpu or 15g (t4?) gpu and this svox2 variant needs at least 27g ram and at least 16g gpu  for fitting 512 voxel size  

<br>
Datasets:
NeRF-synthetic and front facing llff: 
https://drive.google.com/drive/folders/128yBriW1IG_3NJ5Rp7APSTZsJqdJdfc1  
Processed Tanks and temples dataset (with background): 
https://drive.google.com/file/d/1PD4oTP4F8jTtpjd_AQjCsL4h8iYFCyvO  
Real Lego capture: 
https://drive.google.com/file/d/1PG-KllCv4vSRPO7n5lpBjyTjlUyT8Nag  
Pretrained checkpoints (good if you cant train?): 
https://drive.google.com/drive/folders/1SOEJDw8mot7kf5viUK9XryOAmZGe_vvE  



In [ ]:
# Sadly. my 3080 while it is more than twice as fast as Colab Pro, it is still sitting idle due to just 10g gpu mem ending training attempts with out of gpu mem errors
# So let's see what gpu colab given us in cloud (16g gpu mem is minumum for M60 tank sample dataset training and is given on Colab Pro instances).
from psutil import virtual_memory

gpuname = !nvidia-smi -L
gpuname = str(gpuname).replace('PCIE-','').split('Tesla ')[1].split(' ')[0]

print(gpuname)

gpu_info = !nvidia-smi 
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('ERROR!!! Not connected to a GPU. Nothing will work')
else:
  gpu_mem = int(str(gpu_info).split('MiB / ')[1].split('MiB')[0])
  print('You got '+str(gpu_mem)+' gpu mem ')
  if gpu_mem < 16000:
    print(gpu_info,'\n\nBEWARE!!! GPUs with less than 16g will fail with out of memory error on M60 dataset or bigger near training end.')
  else:
    print('which seems enough.')

ram_gb = virtual_memory().total / 1e9

if ram_gb < 20:
  print('BEWARE!!! Not using a high-RAM runtime of Colab Pro. M60 training will fail on out of system memory which is just 12 g on Colab Free')
else:
  print('Great. You are using a high-RAM runtime '+str(int(ram_gb))+'Gb ! 27g is needed to train example M60 Tank dataset\n')

# one day colab will hopefully move to python 3.8 and higher
!python --version

# for faster compiles later
!apt install -y -qq  ninja-build  &> /dev/null

import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
%env MAX_JOBS={cores}

# make sure that colmap is ready if needed to work on our own images later
!apt install colmap &> /dev/null
colmap_version = !colmap -h
print (colmap_version[0])

# this is key to fit in gpu mem. helps to lower gpu fragmentation alloc waste and in turn allows fitting 16g gpu mem typically present in cloud gpus as max
# ie I was last time 20mb short instead of 2g short on gpu mem
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:21

#comment capture out to see cell outputs first time. I personally consider it unecessary super long distracting spam afterwards and cell execution done checkmark is all I need.
#%%capture 

import os
%cd '/content'
if not os.path.exists('svox2'):
  !git clone https://github.com/sxyu/svox2.git  &> /dev/null

  # patch and make py sources again compatible with CoLab (python 3.7) 
  # by removing completely unnecesary 3.8 specific syntaxctic sugar from two lines
  !sed -E -i "s/\{minv=:/minv=\{minv:/g"    /content/svox2/opt/opt.py 
  !sed -E -i "s/\{meanv=:/meanv=\{meanv:/g" /content/svox2/opt/opt.py 
  !sed -E -i "s/\{maxv=:/maxv=\{maxv:/g"    /content/svox2/opt/opt.py 
  !sed -E -i "s/\{minv=:/minv=\{minv:/g"    /content/svox2/opt/render_imgs.py 
  !sed -E -i "s/\{meanv=:/meanv=\{meanv:/g" /content/svox2/opt/render_imgs.py 
  !sed -E -i "s/\{maxv=:/maxv=\{maxv:/g"    /content/svox2/opt/render_imgs.py 

%cd svox2

# There is no point in slow conda and multi env dance in this throwavay colab env anyway plus most packages from environment.yml are installed already.
# So if we stick to colabs py 3.7 then biggest and most complex ones like pytorch cudatoolkit and most of environment.yml requirements is already present 
# and we need to install just 4 missing packages. 
# In short. By sticking to py 3.7 this colab can be restarted from factory reset state very fast without endless and useless huge long reinstalls

!pip install imageio-ffmpeg &> /dev/null
!pip install ipdb           &> /dev/null
!pip install lpips          &> /dev/null
!pip install pymcubes       &> /dev/null

In [ ]:
# To make env restarts even faster. It is perhaps good idea to use gdrive to cache once compiled pytorch wheel and later even checkpoint after training 
# so next time colab runtime resets and forgets all files next bootstrap is fast

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Because compile and install of pytorch wheel is soooo sloooow. We compile it just once first time we run this colab,
# And then cache it on gdrive so next time we can do just 2s fast reinstall on each runtime restart instead of 20 min whl compile
# But. Because we can get different gpu next time. We need to compile and cache multiple whl files for each  gpu we encounter and save whls to gdrive in dirs with gpu names

import os
from google.colab import files
%cd /content/svox2

# grab currenly assigned gpu model name (P100-16gb etc.)
gpuname = !nvidia-smi -L
gpuname = str(gpuname).replace('PCIE-','').split('Tesla ')[1].split(' ')[0]
whlname = 'svox2-0.0.1.dev0+sphtexcub.lincolor.fast-cp37-cp37m-linux_x86_64.whl'
whlpath = '/content/drive/MyDrive/'+gpuname+'/'+whlname
print(gpuname)

# compile this whl just once first time to obtain and cache it on gdrive
if not os.path.exists(whlpath):
  !apt install ninja-build
  %env MAX_JOBS=4
  !python setup.py bdist_wheel &> /dev/null
  !mkdir /content/drive/MyDrive/{gpuname}
  !cp ./dist/{whlname} {whlpath}

# install cached whl next time env is reset to skip costly recompilation
if os.path.exists(whlpath):
  !pip install {whlpath} --force-reinstall

In [ ]:
dataset_name = 'TanksAndTempleBG'
experiment   = 'M60'

In [ ]:
# download super cool tank dataset if its dir is not already present
import os
%cd /content/svox2
# Datasets: from gdrive folder where id is last folder name in url: https://drive.google.com/drive/folders/128yBriW1IG_3NJ5Rp7APSTZsJqdJdfc1
gdrive_ids={'TanksAndTempleBG':'1PD4oTP4F8jTtpjd_AQjCsL4h8iYFCyvO', #
            'nerf_llff_data'  :'16VnMcF1KJYxN9QId6TClMsZRahHNMW5g',
            'nerf_real_360'   :'1jzggQ7IPaJJTKx9yLASWHrX8dXHnG5eB',
            'nerf_synthetic'  :'18JxhpWD-4ZmuFKLzKlAw-w5PpzZxXOcG'}

if not os.path.exists('/content/svox2/data/'+dataset_name):
  console_output = !gdown --id {gdrive_ids[dataset_name]} 
  downloaded_filename = str(console_output).split('To: /content/svox2/')[1].split('\'')[0]

  !mkdir data  &> /dev/null
  if '.zip'    in downloaded_filename:
    !unzip  -q   {downloaded_filename} -d data
  
  if '.tar.gz' in downloaded_filename:
    !tar   -xf   {downloaded_filename} -C data

  # if needed, unify root data dir name and subdir structure since some tar based datasets 
  # are in aditional subdirs with unique names. but let's have just one root dir named "data"
  if os.path.exists('/content/svox2/data/'+dataset_name):
    !mv data/{dataset_name}/* data/

  # remove huge downloaded no longer needed file. 
  !rm  -f  {downloaded_filename}

In [ ]:
# TO DO: 
# enable this optional step to upload and colab calibrate your own images 
# and prepare dataset for svox2 to extract voxels via svox2
# 
# %cd /content/svox2/opt/

# !bash scripts/proc_colmap.sh video_to_obj

# !python scripts/run_colmap.py video_to_obj
# !python colmap2nsvf.py video_to_obj/sparse/0
# !python create_split.py -y video_to_obj
# !pip install nerfvis
# !python scripts/view_data.py video_to_obj

In [ ]:
# launch actual training (remember. 27g ram and 16g gpu for TanksAndTempleBG M60 dataset is needed if selected above)
%cd /content/svox2/opt
custom_configs={'TanksAndTempleBG':'tnt',
                'nerf_llff_data'  :'llff',
                'nerf_real_360'   :'llff',
                'nerf_synthetic'  :'syn'}
!./launch.sh {experiment} 0 ../data/{experiment} -c configs/{custom_configs[dataset_name]}.json

In [ ]:
# since training runs detached we can peek at output on demand here when needed
#
# STOP THIS MANUALLY when training is done!!! when you see "* Final eval and save" line in output)
# tail -f on itself will never end!!! 
%cd /content/svox2/opt
!tail -f ckpt/{experiment}/log

In [ ]:
# lets see our final resulting trained checkpoint file size when done . official one is 4G
!ls -la /content/svox2/opt/ckpt/{experiment}/ckpt.npz

In [ ]:
# backup our expensively trained experiment checkpoint to gdrive for future rendering tests even outside of colab if needed 
# or restore from gdrive if training pass above failed or was skipped
import os
%cd /content/svox2/opt
!mkdir /content/drive/MyDrive/ckpt &> /dev/null

# if training was succesfull and produced npz
if os.path.exists('/content/svox2/opt/ckpt/'+experiment+'/ckpt.npz'):
  print('backing ckpt '+experiment+' to gdrive')
  !cp -rf /content/svox2/opt/ckpt/{experiment} /content/drive/MyDrive/ckpt

# if not, or was skipped, let's restore checkpoint of this experiment from our gdrive if any is found
else:
  if os.path.exists('/content/drive/MyDrive/ckpt/'+experiment+'/ckpt.npz'):
    print('loading ckpt '+experiment+' from gdrive')
    !cp -rf /content/drive/MyDrive/ckpt/{experiment} /content/svox2/opt/ckpt

In [ ]:
# (Sadly even thou colab pro has 16gb, it is still just 16g gpu)
# SO. If the damn training for too large datasets keeps failing with: out of memory 
#
# Then. Luckily. We can still at least try download already trained checkpoints (thx. to paper authors ;D yay)
# And at least start playing with rendering images from various angles
#
# BUT BEWARE this is 11g gz download if enabled
# AND ALSO: if 3-4 downloads happen per 24h of this 11g large file from gdrive, which is highly plausible given usage in this colab. 
# Then google colab will block further downloads via this gdown api at this particular day for 24h
# If that happens. You can still supposedly download it via browser and curl/wget or upload it here by other means I guess?

%cd /content/svox2/opt
import os

# change this to True to enable pretrained checkpoints download. But beware again !!! 11gb file
if False:
  !mv ckpt ckpt_our
  if not os.path.exists('ckpt_tnt.tar.gz'): 
    !gdown --id 1v9xb5Sd3ulofwNUynC71I_fdwnSLnFhS 
  !tar -xvf ckpt_tnt.tar.gz  &> /dev/null
  !rm  -rf  ckpt_tnt.tar.gz # delete this huge 11g file as fast as possible
  !ls -la tnt_equirectlin_fasttv_autoscale/M60/ckpt.npz # check ckpt size for m60. should be around 4g
  !mv tnt_equirectlin_fasttv_autoscale ckpt

In [ ]:
# finally lets synthetize some sample images from various angles (typically 360 orbit around object) from trained checkpoint / model
%cd /content/svox2/opt
!python -u render_imgs.py ckpt/{experiment} ../data/{experiment} 

In [ ]:
# transfer 3mb of jpgs in zip is more practical than 40mb pngs or hard to inspect mp4 with too aggresive compression
%%capture 
%cd /content/svox2/opt/ckpt/{experiment}/test_renders
!for i in *.png; do ffmpeg -i "$i" "${i%.*}.jpg" &> /dev/null ; done 
!find . -type f -iname \*.png -delete
%cd /content/svox2/opt
!zip -rq images.zip /content/svox2/opt/ckpt/{experiment}/test_renders &> /dev/null 

In [ ]:
# copy resulting images to special in html cell visible dir. since html gives bigger output preview flexibility
%%capture 
!cp /content/svox2/opt/ckpt/{experiment}/test_renders/*.* /usr/local/share/jupyter/nbextensions/ &> /dev/null
!cp images.zip /usr/local/share/jupyter/nbextensions/ &> /dev/null

In [ ]:
# show some sample synthetized images
%%html
<a href src='/nbextensions/images.zip' download>Download Images</a><br>
<img width="100%" src='/nbextensions/0000.jpg' />
<img width="100%" src='/nbextensions/0020.jpg' />
<img width="100%" src='/nbextensions/0031.jpg' />